# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 3 from HW3-theoretical. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [573]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [574]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [575]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [576]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.    
    
    """
    #create temporary instance of input graph
    graph_copy = graph.copy()
    
    #create blank lists indicating which nodes in original graph are sources and sinks
    #also keep tally of total demand for both sources and sinks
    real_sources = []
    real_sinks = []
    total_source_demand = 0
    total_sink_demand = 0
    
    #sources have demand < 0; sinks have demand > 0
    for state in graph_copy.nodes():
        if graph_copy.node[state]["demand"] < 0:
            real_sources.append(state)
            total_source_demand += graph_copy.node[state]["demand"]
        if graph_copy.node[state]["demand"] > 0:
            real_sinks.append(state)
            total_sink_demand += graph_copy.node[state]["demand"]
            
    if total_source_demand + total_sink_demand != 0:
        raise nx.NetworkXUnfeasible("Network Unfeasible")
    
    #create dummy nodes allowing problem to be reduced to max flow
    graph_copy.add_node("dummy_source")
    graph_copy.add_node("dummy_sink")
    
    #add edges from dummy source to real sources and from real sinks to dummy sink
    for source in real_sources:
        graph_copy.add_edge("dummy_source", source)
        
    for sink in real_sinks:
        graph_copy.add_edge(sink, "dummy_sink")
    
    for (s1, s2) in graph_copy.edges():
        if s2 == "dummy_sink":
            graph_copy.edge[s1][s2]['capacity'] = 1
            
    #get maxflow and flow along each edge        
    maxflow, flow_dict = nx.maximum_flow(graph_copy, 'dummy_source', 'dummy_sink')
    
    #necessary condition
    if maxflow < total_sink_demand:
        raise nx.NetworkXUnfeasible("Network Unfeasible") 
    
    #create new dict, removing dummy_source and dummy_sink from results
    flow_dict_clean = {}
    
    for state1, block2 in flow_dict.iteritems():
        if state1 != 'dummy_source' and state1 != 'dummy_sink':
            for state2, edge_flow in block2.iteritems():
                flow_dict_clean[state1] = {state2:flow_dict[state1][state2] for state2 in flow_dict[state1] if (state2 != 'dummy_source' and state2 != 'dummy_sink')}
        
    return flow_dict_clean

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [571]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    
    net_flow = {}

    #If a state is listed first in an edge, subtract that flow from its net flow. If listed second, add to net flow.
    for state1, block2 in flow.iteritems():
        for state2, edge_flow in block2.iteritems():
            if state1 not in net_flow:
                net_flow[state1] = -flow[state1][state2]
            else:
                net_flow[state1] = net_flow[state1] - flow[state1][state2]
            if state2 not in net_flow:
                net_flow[state2] = flow[state1][state2]
            else:
                net_flow[state2] = net_flow[state2] + flow[state1][state2]
            
    
    return net_flow

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [572]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True


In [578]:
div

{'AL': 1,
 'AR': 1,
 'AZ': 1,
 'CA': -48,
 'CO': 1,
 'CT': 1,
 'DC': 1,
 'DE': 1,
 'FL': 1,
 'GA': 1,
 'IA': 1,
 'ID': 1,
 'IL': 1,
 'IN': 1,
 'KS': 1,
 'KY': 1,
 'LA': 1,
 'MA': 1,
 'MD': 1,
 'ME': 1,
 'MI': 1,
 'MN': 1,
 'MO': 1,
 'MS': 1,
 'MT': 1,
 'NC': 1,
 'ND': 1,
 'NE': 1,
 'NH': 1,
 'NJ': 1,
 'NM': 1,
 'NV': 1,
 'NY': 1,
 'OH': 1,
 'OK': 1,
 'OR': 1,
 'PA': 1,
 'RI': 1,
 'SC': 1,
 'SD': 1,
 'TN': 1,
 'TX': 1,
 'UT': 1,
 'VA': 1,
 'VT': 1,
 'WA': 1,
 'WI': 1,
 'WV': 1,
 'WY': 1}